Finding what has changed between two seemingly identical spreadsheets

In [ ]:
%autosave 30

import pandas as pd
import numpy as np
import xlrd

# Define the diff function to show the changes in each field
def report_diff(x):
    return x[0] if x[0] == x[1] else '{} ---> {}'.format(*x)

# Read in the two files but call the data old and new and create columns to track
old = pd.read_excel('sample-address-old.xlsx', 'Sheet1', na_values=['NA'])
new = pd.read_excel('sample-address-new.xlsx', 'Sheet1', na_values=['NA'])
old['version'] = "old"
new['version'] = "new"

#Join all the data together and ignore indexes so it all gets added
full_set = pd.concat([old,new],ignore_index=True)

# Let's see what changes in the main columns we care about
changes = full_set.drop_duplicates(subset=["account number","name","street","city","state","postal code"],keep='first')

#We want to know where the duplicate account numbers are, that means there have been changes
dupe_accts = changes.set_index('account number').index.get_duplicates()

#Get all the duplicate rows
dupes = changes[changes["account number"].isin(dupe_accts)]

#Pull out the old and new data into separate dataframes
change_new = dupes[(dupes["version"] == "new")]
change_old = dupes[(dupes["version"] == "old")]

#Drop the temp columns - we don't need them now
change_new = change_new.drop(['version'], axis=1)
change_old = change_old.drop(['version'], axis=1)

#Index on the account numbers
change_new.set_index('account number',inplace=True)
change_old.set_index('account number',inplace=True)

#Now we can diff because we have two data sets of the same size with the same index
diff_panel = pd.Panel(dict(df1=change_old,df2=change_new))
diff_output = diff_panel.apply(report_diff, axis=0)

#Diff'ing is done, we need to get a list of removed items

#Flag all duplicated account numbers
changes['duplicate']=changes["account number"].isin(dupe_accts)

#Identify non-duplicated items that are in the old version and did not show in the new version
removed_accounts = changes[(changes["duplicate"] == False) & (changes["version"] == "old")]

# We have the old and diff, we need to figure out which ones are new

#Drop duplicates but keep the first item instead of the last
new_account_set = full_set.drop_duplicates(subset=["account number","name","street","city","state","postal code"],keep='first')

#Identify dupes in this new dataframe
new_account_set['duplicate']=new_account_set["account number"].isin(dupe_accts)

#Identify added accounts
added_accounts = new_account_set[(new_account_set["duplicate"] == False) & (new_account_set["version"] == "new")]

#Save the changes to excel but only include the columns we care about
writer = pd.ExcelWriter("my-diff-2.xlsx")
diff_output.to_excel(writer,"changed")
removed_accounts.to_excel(writer,"removed",index=False,columns=["account number",
                                         "name","street","city","state","postal code"])
added_accounts.to_excel(writer,"added",index=False,columns=["account number",
                                         "name","street","city","state","postal code"])
writer.save()

Common Excel tasks

In [41]:
import pandas as pd
import numpy as np
df = pd.read_excel("excel-comp-data.xlsx")
df.head()

,account,name,street,city,state,postal-code,Jan,Feb,Mar
0,211829,"Kerluke, Koepp and Hilpert",34456 Sean Highway,New Jaycob,Texas,28752,10000,62000,35000
1,320563,Walter-Trantow,1311 Alvis Tunnel,Port Khadijah,NorthCarolina,38365,95000,45000,35000
2,648336,"Bashirian, Kunde and Price",62184 Schamberger Underpass Apt. 231,New Lilianland,Iowa,76517,91000,120000,35000
3,109996,"D'Amore, Gleichner and Bode",155 Fadel Crescent Apt. 144,Hyattburgh,Maine,46021,45000,120000,10000
4,121213,Bauch-Goldner,7274 Marissa Common,Shanahanchester,California,49681,162000,120000,35000


Add a totals column for Jan, Feb, Mar, like Sum(A+B+C)

In [42]:
df["total"] = df["Jan"] + df["Feb"] + df["Mar"]
df.head()

,account,name,street,city,state,postal-code,Jan,Feb,Mar,total
0,211829,"Kerluke, Koepp and Hilpert",34456 Sean Highway,New Jaycob,Texas,28752,10000,62000,35000,107000
1,320563,Walter-Trantow,1311 Alvis Tunnel,Port Khadijah,NorthCarolina,38365,95000,45000,35000,175000
2,648336,"Bashirian, Kunde and Price",62184 Schamberger Underpass Apt. 231,New Lilianland,Iowa,76517,91000,120000,35000,246000
3,109996,"D'Amore, Gleichner and Bode",155 Fadel Crescent Apt. 144,Hyattburgh,Maine,46021,45000,120000,10000,175000
4,121213,Bauch-Goldner,7274 Marissa Common,Shanahanchester,California,49681,162000,120000,35000,317000


Now let's do some column level analysis

In [5]:
df["Jan"].sum(), df["Jan"].mean(),df["Jan"].min(),df["Jan"].max()

(1462000, 97466.66666666667, 10000, 162000)

To add a total column sum in a row, it takes a few more steps in a data frame

In [47]:
# create the sums in the sum_row object
sum_row=df[["Jan","Feb","Mar","total"]].sum()

# transpose and store in new df
df_sum=pd.DataFrame(data=sum_row).T

#The final thing we need to do before adding the totals back 
#is to add the missing columns. We use reindex to do this for us. 
#The trick is to add all of our columns and then allow pandas to fill in the values that are missing.
df_sum=df_sum.reindex(columns=df.columns)
df_sum

#append that df to the original
df_final=df.append(df_sum,ignore_index=True)

#check and see how it looks
df_final.tail()

,account,name,street,city,state,postal-code,Jan,Feb,Mar,total
11,231907.0,Hahn-Moore,18115 Olivine Throughway,Norbertomouth,NorthDakota,31415.0,150000,10000,162000,322000
12,242368.0,"Frami, Anderson and Donnelly",182 Bertie Road,East Davian,Iowa,72686.0,162000,120000,35000,317000
13,268755.0,Walsh-Haley,2624 Beatty Parkways,Goodwinmouth,RhodeIsland,31919.0,55000,120000,35000,210000
14,273274.0,McDermott PLC,8917 Bergstrom Meadow,Kathryneborough,Delaware,27933.0,150000,120000,70000,340000
15,NaN,NaN,NaN,NaN,NaN,NaN,1462000,1507000,717000,3686000


For another example, let’s try to add a state abbreviation to the data set. From an Excel perspective the easiest way is probably to add a new column, do a vlookup on the state name and fill in the abbreviation. Problems arise if mispellings in lookup table

In [7]:
!pip install fuzzywuzzy

We need to import fuzzywuzzy functions. The other piece of code we need is a state name to abbreviation mapping. Instead of trying to type it myself, a little googling found this code.

In [25]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
state_to_code = {"VERMONT": "VT", "GEORGIA": "GA", "IOWA": "IA", "Armed Forces Pacific": "AP", "GUAM": "GU",
                 "KANSAS": "KS", "FLORIDA": "FL", "AMERICAN SAMOA": "AS", "NORTH CAROLINA": "NC", "HAWAII": "HI",
                 "NEW YORK": "NY", "CALIFORNIA": "CA", "ALABAMA": "AL", "IDAHO": "ID", "FEDERATED STATES OF MICRONESIA": "FM",
                 "Armed Forces Americas": "AA", "DELAWARE": "DE", "ALASKA": "AK", "ILLINOIS": "IL",
                 "Armed Forces Africa": "AE", "SOUTH DAKOTA": "SD", "CONNECTICUT": "CT", "MONTANA": "MT", "MASSACHUSETTS": "MA",
                 "PUERTO RICO": "PR", "Armed Forces Canada": "AE", "NEW HAMPSHIRE": "NH", "MARYLAND": "MD", "NEW MEXICO": "NM",
                 "MISSISSIPPI": "MS", "TENNESSEE": "TN", "PALAU": "PW", "COLORADO": "CO", "Armed Forces Middle East": "AE",
                 "NEW JERSEY": "NJ", "UTAH": "UT", "MICHIGAN": "MI", "WEST VIRGINIA": "WV", "WASHINGTON": "WA",
                 "MINNESOTA": "MN", "OREGON": "OR", "VIRGINIA": "VA", "VIRGIN ISLANDS": "VI", "MARSHALL ISLANDS": "MH",
                 "WYOMING": "WY", "OHIO": "OH", "SOUTH CAROLINA": "SC", "INDIANA": "IN", "NEVADA": "NV", "LOUISIANA": "LA",
                 "NORTHERN MARIANA ISLANDS": "MP", "NEBRASKA": "NE", "ARIZONA": "AZ", "WISCONSIN": "WI", "NORTH DAKOTA": "ND",
                 "Armed Forces Europe": "AE", "PENNSYLVANIA": "PA", "OKLAHOMA": "OK", "KENTUCKY": "KY", "RHODE ISLAND": "RI",
                 "DISTRICT OF COLUMBIA": "DC", "ARKANSAS": "AR", "MISSOURI": "MO", "TEXAS": "TX", "MAINE": "ME"}

In [26]:
# How fuzzy text matching works

process.extractOne("Minnesotta",choices=state_to_code.keys())

('MINNESOTA', 95)

We use the 80 score_cutoff for this data. You can play with it to see what number works for your data. You’ll notice that we either return a valid abbreviation or an np.nan so that we have some valid values in the field.

In [48]:

# we define our state conversion function
def convert_state(row):
    abbrev = process.extractOne(row["state"],choices=state_to_code.keys(),score_cutoff=80)
    if abbrev:
        return state_to_code[abbrev[0]]
    return np.nan

# we add our new column, select its position, title and fill with  NA values
df_final.insert(6, "abbrev", np.nan)
df_final.head()


,account,name,street,city,state,postal-code,abbrev,Jan,Feb,Mar,total
0,211829.0,"Kerluke, Koepp and Hilpert",34456 Sean Highway,New Jaycob,Texas,28752.0,NaN,10000,62000,35000,107000
1,320563.0,Walter-Trantow,1311 Alvis Tunnel,Port Khadijah,NorthCarolina,38365.0,NaN,95000,45000,35000,175000
2,648336.0,"Bashirian, Kunde and Price",62184 Schamberger Underpass Apt. 231,New Lilianland,Iowa,76517.0,NaN,91000,120000,35000,246000
3,109996.0,"D'Amore, Gleichner and Bode",155 Fadel Crescent Apt. 144,Hyattburgh,Maine,46021.0,NaN,45000,120000,10000,175000
4,121213.0,Bauch-Goldner,7274 Marissa Common,Shanahanchester,California,49681.0,NaN,162000,120000,35000,317000


In [49]:
# replace Nan with str or fuzzywuzzy won't work on final line for some reason
df_final = df_final.replace(np.nan, 'blank', regex=True)

In [50]:
# apply our function and look at the results
df_final['abbrev'] = df_final.apply(convert_state, axis=1)
df_final.tail()

,account,name,street,city,state,postal-code,abbrev,Jan,Feb,Mar,total
11,231907,Hahn-Moore,18115 Olivine Throughway,Norbertomouth,NorthDakota,31415,ND,150000,10000,162000,322000
12,242368,"Frami, Anderson and Donnelly",182 Bertie Road,East Davian,Iowa,72686,IA,162000,120000,35000,317000
13,268755,Walsh-Haley,2624 Beatty Parkways,Goodwinmouth,RhodeIsland,31919,RI,55000,120000,35000,210000
14,273274,McDermott PLC,8917 Bergstrom Meadow,Kathryneborough,Delaware,27933,DE,150000,120000,70000,340000
15,blank,blank,blank,blank,blank,blank,NaN,1462000,1507000,717000,3686000


For the final section of this article, let’s get some monthly subtotals by state.

In [51]:
df_sub=df_final[["abbrev","Jan","Feb","Mar","total"]].groupby('abbrev').sum()
df_sub

,Jan,Feb,Mar,total
abbrev,,,,
AR,150000,120000,35000,305000
CA,162000,120000,35000,317000
DE,150000,120000,70000,340000
IA,253000,240000,70000,563000
ID,70000,120000,35000,225000
ME,45000,120000,10000,175000
MS,62000,120000,70000,252000
NC,95000,45000,35000,175000
ND,150000,10000,162000,322000


Let's format those as money

In [53]:
def money(x):
    return "£{:,.0f}".format(x)

formatted_df = df_sub.applymap(money)
formatted_df

,Jan,Feb,Mar,total
abbrev,,,,
AR,"£150,000","£120,000","£35,000","£305,000"
CA,"£162,000","£120,000","£35,000","£317,000"
DE,"£150,000","£120,000","£70,000","£340,000"
IA,"£253,000","£240,000","£70,000","£563,000"
ID,"£70,000","£120,000","£35,000","£225,000"
ME,"£45,000","£120,000","£10,000","£175,000"
MS,"£62,000","£120,000","£70,000","£252,000"
NC,"£95,000","£45,000","£35,000","£175,000"
ND,"£150,000","£10,000","£162,000","£322,000"


In [54]:
# we can add totals again like we did before
sum_row=df_sub[["Jan","Feb","Mar","total"]].sum()

# transpose and make new df
df_sub_sum=pd.DataFrame(data=sum_row).T
df_sub_sum=df_sub_sum.applymap(money)

# append etc
final_table = formatted_df.append(df_sub_sum)

# change indexed title of final row from 0 to total
final_table = final_table.rename(index={0:"Total"})
final_table

,Jan,Feb,Mar,total
AR,"£150,000","£120,000","£35,000","£305,000"
CA,"£162,000","£120,000","£35,000","£317,000"
DE,"£150,000","£120,000","£70,000","£340,000"
IA,"£253,000","£240,000","£70,000","£563,000"
ID,"£70,000","£120,000","£35,000","£225,000"
ME,"£45,000","£120,000","£10,000","£175,000"
MS,"£62,000","£120,000","£70,000","£252,000"
NC,"£95,000","£45,000","£35,000","£175,000"
ND,"£150,000","£10,000","£162,000","£322,000"
PA,"£70,000","£95,000","£35,000","£200,000"


Now lets try some other Excel type tasks - check the data types on this sheet

In [55]:
import pandas as pd
import numpy as np
df = pd.read_excel("sample-salesv3.xlsx")
df.dtypes

account number      int64
name               object
sku                object
quantity            int64
unit price        float64
ext price         float64
date               object
dtype: object

In [56]:
# the date column is an object so can be converted
df['date'] = pd.to_datetime(df['date'])
df.dtypes

account number             int64
name                      object
sku                       object
quantity                   int64
unit price               float64
ext price                float64
date              datetime64[ns]
dtype: object

You can do various type of filtering:

In [57]:
df[df["account number"]==307599].head()

,account number,name,sku,quantity,unit price,ext price,date
3,307599,"Kassulke, Ondricka and Metz",S1-65481,41,21.05,863.05,2014-01-01 15:05:22
13,307599,"Kassulke, Ondricka and Metz",S2-10342,17,12.44,211.48,2014-01-04 07:53:01
34,307599,"Kassulke, Ondricka and Metz",S2-78676,35,33.04,1156.40,2014-01-10 05:26:31
58,307599,"Kassulke, Ondricka and Metz",B1-20000,22,37.87,833.14,2014-01-15 16:22:22
70,307599,"Kassulke, Ondricka and Metz",S2-10342,44,96.79,4258.76,2014-01-18 06:32:31


In [58]:
df[df["quantity"] > 22].head()

,account number,name,sku,quantity,unit price,ext price,date
0,740150,Barton LLC,B1-20000,39,86.69,3380.91,2014-01-01 07:21:51
2,218895,Kulas Inc,B1-69924,23,90.70,2086.10,2014-01-01 13:24:58
3,307599,"Kassulke, Ondricka and Metz",S1-65481,41,21.05,863.05,2014-01-01 15:05:22
14,737550,"Fritsch, Russel and Anderson",B1-53102,23,71.56,1645.88,2014-01-04 08:57:48
15,239344,Stokes LLC,S1-06532,34,71.51,2431.34,2014-01-04 11:34:58


In [59]:
#If we want to do more complex filtering, we can use map to filter on various criteria. 
# In this example, let’s look for items with sku’s that start with B1.

df[df["sku"].map(lambda x: x.startswith('B1'))].head()

,account number,name,sku,quantity,unit price,ext price,date
0,740150,Barton LLC,B1-20000,39,86.69,3380.91,2014-01-01 07:21:51
2,218895,Kulas Inc,B1-69924,23,90.70,2086.10,2014-01-01 13:24:58
6,218895,Kulas Inc,B1-65551,2,31.10,62.20,2014-01-02 10:57:23
14,737550,"Fritsch, Russel and Anderson",B1-53102,23,71.56,1645.88,2014-01-04 08:57:48
17,239344,Stokes LLC,B1-50809,14,16.23,227.22,2014-01-04 22:14:32


In [60]:
# or chain multiple filters together with &

df[df["sku"].map(lambda x: x.startswith('B1')) & (df["quantity"] > 22)].head()

,account number,name,sku,quantity,unit price,ext price,date
0,740150,Barton LLC,B1-20000,39,86.69,3380.91,2014-01-01 07:21:51
2,218895,Kulas Inc,B1-69924,23,90.70,2086.10,2014-01-01 13:24:58
14,737550,"Fritsch, Russel and Anderson",B1-53102,23,71.56,1645.88,2014-01-04 08:57:48
26,737550,"Fritsch, Russel and Anderson",B1-53636,42,42.06,1766.52,2014-01-08 00:02:11
31,714466,Trantow-Barrows,B1-33087,32,19.56,625.92,2014-01-09 10:16:32


In [61]:
# isin function looks for multiple values

df[df["account number"].isin([714466,218895])].head()

,account number,name,sku,quantity,unit price,ext price,date
1,714466,Trantow-Barrows,S2-77896,-1,63.16,-63.16,2014-01-01 10:00:47
2,218895,Kulas Inc,B1-69924,23,90.70,2086.10,2014-01-01 13:24:58
5,714466,Trantow-Barrows,S2-77896,17,87.63,1489.71,2014-01-02 10:07:15
6,218895,Kulas Inc,B1-65551,2,31.10,62.20,2014-01-02 10:57:23
8,714466,Trantow-Barrows,S1-50961,22,84.09,1849.98,2014-01-03 11:29:02


In [63]:
# or you can use query function
df.query('name == ["Kulas Inc","Barton LLC"]').head()

,account number,name,sku,quantity,unit price,ext price,date
0,740150,Barton LLC,B1-20000,39,86.69,3380.91,2014-01-01 07:21:51
2,218895,Kulas Inc,B1-69924,23,90.70,2086.10,2014-01-01 13:24:58
6,218895,Kulas Inc,B1-65551,2,31.10,62.20,2014-01-02 10:57:23
33,218895,Kulas Inc,S1-06532,3,22.36,67.08,2014-01-09 23:58:27
36,218895,Kulas Inc,S2-34077,16,73.04,1168.64,2014-01-10 12:07:30


In [65]:
# when working with dates it is advisable to sort them first
df = df.sort_values('date')
df.head()

,account number,name,sku,quantity,unit price,ext price,date
0,740150,Barton LLC,B1-20000,39,86.69,3380.91,2014-01-01 07:21:51
1,714466,Trantow-Barrows,S2-77896,-1,63.16,-63.16,2014-01-01 10:00:47
2,218895,Kulas Inc,B1-69924,23,90.70,2086.10,2014-01-01 13:24:58
3,307599,"Kassulke, Ondricka and Metz",S1-65481,41,21.05,863.05,2014-01-01 15:05:22
4,412290,Jerde-Hilpert,S2-34077,6,83.21,499.26,2014-01-01 23:26:55


In [66]:
# you can filter:
df[df['date'] >='20140905'].head()

,account number,name,sku,quantity,unit price,ext price,date
1042,163416,Purdy-Kunde,B1-38851,41,98.69,4046.29,2014-09-05 01:52:32
1043,714466,Trantow-Barrows,S1-30248,1,37.16,37.16,2014-09-05 06:17:19
1044,729833,Koepp Ltd,S1-65481,48,16.04,769.92,2014-09-05 08:54:41
1045,729833,Koepp Ltd,S2-11481,6,26.50,159.00,2014-09-05 16:33:15
1046,737550,"Fritsch, Russel and Anderson",B1-33364,4,76.44,305.76,2014-09-06 08:59:08


In [67]:
# we can also do partial filtering for say after a certain month
df[df['date'] >='2014-03'].head()

,account number,name,sku,quantity,unit price,ext price,date
242,163416,Purdy-Kunde,S1-30248,19,65.03,1235.57,2014-03-01 16:07:40
243,527099,Sanford and Sons,S2-82423,3,76.21,228.63,2014-03-01 17:18:01
244,527099,Sanford and Sons,B1-50809,8,70.78,566.24,2014-03-01 18:53:09
245,737550,"Fritsch, Russel and Anderson",B1-50809,20,50.11,1002.20,2014-03-01 23:47:17
246,688981,Keeling LLC,B1-86481,-1,97.16,-97.16,2014-03-02 01:46:44


In [68]:
# and chain criteria
df[(df['date'] >='20140701') & (df['date'] <= '20140715')].head()

,account number,name,sku,quantity,unit price,ext price,date
778,737550,"Fritsch, Russel and Anderson",S1-65481,35,70.51,2467.85,2014-07-01 00:21:58
779,218895,Kulas Inc,S1-30248,9,16.56,149.04,2014-07-01 00:52:38
780,163416,Purdy-Kunde,S2-82423,44,68.27,3003.88,2014-07-01 08:15:52
781,672390,Kuhn-Gusikowski,B1-04202,48,99.39,4770.72,2014-07-01 11:12:13
782,642753,Pollich LLC,S2-23246,1,51.29,51.29,2014-07-02 04:02:39


In [70]:
# you can express dates in multiple formats and it still understands
df[df['date'] >= 'Oct-2014'].head()

# or

df[df['date'] >= '10-10-2014'].head()

,account number,name,sku,quantity,unit price,ext price,date
1174,257198,"Cronin, Oberbrunner and Spencer",S2-34077,13,12.24,159.12,2014-10-10 02:59:06
1175,740150,Barton LLC,S1-65481,28,53.00,1484.00,2014-10-10 15:08:53
1176,146832,Kiehn-Spinka,S1-27722,15,64.39,965.85,2014-10-10 18:24:01
1177,257198,"Cronin, Oberbrunner and Spencer",S2-16558,3,35.34,106.02,2014-10-11 01:48:13
1178,737550,"Fritsch, Russel and Anderson",B1-53636,10,56.95,569.50,2014-10-11 10:25:53


When working with time series data, if we convert the data to use the date as as the index, we can do some more filtering variations.

In [71]:
# we set the new index
df2 = df.set_index(['date'])
df2.head()

,account number,name,sku,quantity,unit price,ext price
date,,,,,,
2014-01-01 07:21:51,740150,Barton LLC,B1-20000,39,86.69,3380.91
2014-01-01 10:00:47,714466,Trantow-Barrows,S2-77896,-1,63.16,-63.16
2014-01-01 13:24:58,218895,Kulas Inc,B1-69924,23,90.70,2086.10
2014-01-01 15:05:22,307599,"Kassulke, Ondricka and Metz",S1-65481,41,21.05,863.05
2014-01-01 23:26:55,412290,Jerde-Hilpert,S2-34077,6,83.21,499.26


In [73]:
# we can slice to get a range, note we can use any format
df2["2014-Jan-1":"2014-Feb-1"].head()

,account number,name,sku,quantity,unit price,ext price
date,,,,,,
2014-01-01 07:21:51,740150,Barton LLC,B1-20000,39,86.69,3380.91
2014-01-01 10:00:47,714466,Trantow-Barrows,S2-77896,-1,63.16,-63.16
2014-01-01 13:24:58,218895,Kulas Inc,B1-69924,23,90.70,2086.10
2014-01-01 15:05:22,307599,"Kassulke, Ondricka and Metz",S1-65481,41,21.05,863.05
2014-01-01 23:26:55,412290,Jerde-Hilpert,S2-34077,6,83.21,499.26


In [74]:
df2["2014-Jan-1":"2014-Feb-1"].tail()

,account number,name,sku,quantity,unit price,ext price
date,,,,,,
2014-01-31 22:51:18,383080,Will LLC,B1-05914,43,80.17,3447.31
2014-02-01 09:04:59,383080,Will LLC,B1-20000,7,33.69,235.83
2014-02-01 11:51:46,412290,Jerde-Hilpert,S1-27722,11,21.12,232.32
2014-02-01 17:24:32,412290,Jerde-Hilpert,B1-86481,3,35.99,107.97
2014-02-01 19:56:48,412290,Jerde-Hilpert,B1-20000,23,78.90,1814.70


In [75]:
df2["2014"].head()

,account number,name,sku,quantity,unit price,ext price
date,,,,,,
2014-01-01 07:21:51,740150,Barton LLC,B1-20000,39,86.69,3380.91
2014-01-01 10:00:47,714466,Trantow-Barrows,S2-77896,-1,63.16,-63.16
2014-01-01 13:24:58,218895,Kulas Inc,B1-69924,23,90.70,2086.10
2014-01-01 15:05:22,307599,"Kassulke, Ondricka and Metz",S1-65481,41,21.05,863.05
2014-01-01 23:26:55,412290,Jerde-Hilpert,S2-34077,6,83.21,499.26


In [76]:
df2["2014-Dec"].head()

,account number,name,sku,quantity,unit price,ext price
date,,,,,,
2014-12-01 20:15:34,714466,Trantow-Barrows,S1-82801,3,77.97,233.91
2014-12-02 20:00:04,146832,Kiehn-Spinka,S2-23246,37,57.81,2138.97
2014-12-03 04:43:53,218895,Kulas Inc,S2-77896,30,77.44,2323.20
2014-12-03 06:05:43,141962,Herman LLC,B1-53102,20,26.12,522.40
2014-12-03 14:17:34,642753,Pollich LLC,B1-53636,19,71.21,1352.99


In [77]:
# Pandas has support for vectorized string functions as well.

df[df['sku'].str.contains('B1')].head()

,account number,name,sku,quantity,unit price,ext price,date
0,740150,Barton LLC,B1-20000,39,86.69,3380.91,2014-01-01 07:21:51
2,218895,Kulas Inc,B1-69924,23,90.70,2086.10,2014-01-01 13:24:58
6,218895,Kulas Inc,B1-65551,2,31.10,62.20,2014-01-02 10:57:23
14,737550,"Fritsch, Russel and Anderson",B1-53102,23,71.56,1645.88,2014-01-04 08:57:48
17,239344,Stokes LLC,B1-50809,14,16.23,227.22,2014-01-04 22:14:32


In [81]:
# We can string queries together and use sort to control how the data is ordered.

df[(df['sku'].str.contains('B1-531')) & (df['quantity']>40)].sort_values(['quantity','name'],ascending=[0,1])

,account number,name,sku,quantity,unit price,ext price,date
684,642753,Pollich LLC,B1-53102,46,26.07,1199.22,2014-06-08 19:33:33
792,688981,Keeling LLC,B1-53102,45,41.19,1853.55,2014-07-04 21:42:22
176,383080,Will LLC,B1-53102,45,89.22,4014.90,2014-02-11 04:14:09
1213,604255,"Halvorson, Crona and Champlin",B1-53102,41,55.05,2257.05,2014-10-18 19:27:01
1215,307599,"Kassulke, Ondricka and Metz",B1-53102,41,93.70,3841.70,2014-10-18 23:25:10
1128,714466,Trantow-Barrows,B1-53102,41,55.68,2282.88,2014-09-27 10:42:48
1001,424914,White-Trantow,B1-53102,41,81.25,3331.25,2014-08-26 11:44:30


Finding unique values is easy

In [82]:
df["name"].unique()

array(['Barton LLC', 'Trantow-Barrows', 'Kulas Inc',
       'Kassulke, Ondricka and Metz', 'Jerde-Hilpert', 'Koepp Ltd',
       'Fritsch, Russel and Anderson', 'Kiehn-Spinka', 'Keeling LLC',
       'Frami, Hills and Schmidt', 'Stokes LLC', 'Kuhn-Gusikowski',
       'Herman LLC', 'White-Trantow', 'Sanford and Sons', 'Pollich LLC',
       'Will LLC', 'Cronin, Oberbrunner and Spencer',
       'Halvorson, Crona and Champlin', 'Purdy-Kunde'], dtype=object)

In [83]:
# if we wanted the account number for each company we use this:

df.drop_duplicates(subset=["account number","name"]).head()

,account number,name,sku,quantity,unit price,ext price,date
0,740150,Barton LLC,B1-20000,39,86.69,3380.91,2014-01-01 07:21:51
1,714466,Trantow-Barrows,S2-77896,-1,63.16,-63.16,2014-01-01 10:00:47
2,218895,Kulas Inc,B1-69924,23,90.70,2086.10,2014-01-01 13:24:58
3,307599,"Kassulke, Ondricka and Metz",S1-65481,41,21.05,863.05,2014-01-01 15:05:22
4,412290,Jerde-Hilpert,S2-34077,6,83.21,499.26,2014-01-01 23:26:55


In [88]:
# We are obviously pulling in more data than we need and getting some non-useful information, 
# so select only the first and second columns using iloc .

# note that [0,1] specifies the column numbers we want to return, can be anything

df.drop_duplicates(subset=["account number","name"]).iloc[:,[0,1]]

,account number,name
0,740150,Barton LLC
1,714466,Trantow-Barrows
2,218895,Kulas Inc
3,307599,"Kassulke, Ondricka and Metz"
4,412290,Jerde-Hilpert
7,729833,Koepp Ltd
9,737550,"Fritsch, Russel and Anderson"
10,146832,Kiehn-Spinka
11,688981,Keeling LLC
12,786968,"Frami, Hills and Schmidt"


What about combining multiple excel files? We can import glob and use it to pick out the sales files

In [111]:
# import glob and use it to check what sales files we have
import glob

for file in glob.glob("sales*.xlsx"):
    print(file)

sales-feb-2014.xlsx
sales-jan-2014.xlsx
sales-mar-2014.xlsx


In [112]:
# create a new blank data frame and then append data from the 3 sales files

all_data = pd.DataFrame()
for file in glob.glob("sales*.xlsx"):
    df = pd.read_excel(file)
    all_data = all_data.append(df,ignore_index=True)

In [113]:
all_data.describe()

,account number,quantity,unit price,ext price
count,384.000000,384.000000,384.000000,384.000000
mean,478125.989583,24.372396,56.651406,1394.517344
std,220902.947401,14.373219,27.075883,1117.809743
min,141962.000000,-1.000000,10.210000,-97.160000
25%,257198.000000,12.000000,32.612500,482.745000
50%,424914.000000,23.500000,58.160000,1098.710000
75%,714466.000000,37.000000,80.965000,2132.260000
max,786968.000000,49.000000,99.730000,4590.810000


In [114]:
all_data.head()

,account number,name,sku,quantity,unit price,ext price,date
0,383080,Will LLC,B1-20000,7,33.69,235.83,2014-02-01 09:04:59
1,412290,Jerde-Hilpert,S1-27722,11,21.12,232.32,2014-02-01 11:51:46
2,412290,Jerde-Hilpert,B1-86481,3,35.99,107.97,2014-02-01 17:24:32
3,412290,Jerde-Hilpert,B1-20000,23,78.90,1814.70,2014-02-01 19:56:48
4,672390,Kuhn-Gusikowski,S1-06532,48,55.82,2679.36,2014-02-02 03:45:20


In [116]:
# as per best practice we should convert date to date time object

all_data['date'] = pd.to_datetime(all_data['date'])


Combining Data

Now that we have all of the data into one DataFrame, we can do any manipulations the DataFrame supports. In this case, the next thing we want to do is read in another file that contains the customer status by account. You can think of this as a company's customer segmentation strategy or some other mechanism for identifying their customers.

First, we read in the data.


In [118]:
status = pd.read_excel("customer-status.xlsx")
status

,account number,name,status
0,740150,Barton LLC,gold
1,714466,Trantow-Barrows,silver
2,218895,Kulas Inc,bronze
3,307599,"Kassulke, Ondricka and Metz",bronze
4,412290,Jerde-Hilpert,bronze
5,729833,Koepp Ltd,silver
6,146832,Kiehn-Spinka,silver
7,688981,Keeling LLC,silver
8,786968,"Frami, Hills and Schmidt",silver
9,239344,Stokes LLC,gold


We want to merge this data with our concatenated data set of sales. We use panda's merge function and tell it to do a left join which is similar to Excel's vlookup function.

In [119]:
all_data_st = pd.merge(all_data, status, how='left')
all_data_st.head()

,account number,name,sku,quantity,unit price,ext price,date,status
0,383080,Will LLC,B1-20000,7,33.69,235.83,2014-02-01 09:04:59,NaN
1,412290,Jerde-Hilpert,S1-27722,11,21.12,232.32,2014-02-01 11:51:46,bronze
2,412290,Jerde-Hilpert,B1-86481,3,35.99,107.97,2014-02-01 17:24:32,bronze
3,412290,Jerde-Hilpert,B1-20000,23,78.90,1814.70,2014-02-01 19:56:48,bronze
4,672390,Kuhn-Gusikowski,S1-06532,48,55.82,2679.36,2014-02-02 03:45:20,silver


In [120]:
#This looks pretty good but let's look at a specific account.

all_data_st[all_data_st["account number"]==737550].head()

,account number,name,sku,quantity,unit price,ext price,date,status
15,737550,"Fritsch, Russel and Anderson",S1-47412,40,51.01,2040.40,2014-02-05 01:20:40,NaN
25,737550,"Fritsch, Russel and Anderson",S1-06532,34,18.69,635.46,2014-02-07 09:22:02,NaN
66,737550,"Fritsch, Russel and Anderson",S1-27722,15,70.23,1053.45,2014-02-16 18:24:42,NaN
78,737550,"Fritsch, Russel and Anderson",S2-34077,26,93.35,2427.10,2014-02-20 18:45:43,NaN
80,737550,"Fritsch, Russel and Anderson",S1-93683,31,10.52,326.12,2014-02-21 13:55:45,NaN


This account number was not in our status file, so we have a bunch of NaN's. We can decide how we want to handle this situation. For this specific case, let's label all missing accounts as bronze. Use the fillna function to easily accomplish this on the status column.

In [121]:
all_data_st['status'].fillna('bronze',inplace=True)
all_data_st.head()

,account number,name,sku,quantity,unit price,ext price,date,status
0,383080,Will LLC,B1-20000,7,33.69,235.83,2014-02-01 09:04:59,bronze
1,412290,Jerde-Hilpert,S1-27722,11,21.12,232.32,2014-02-01 11:51:46,bronze
2,412290,Jerde-Hilpert,B1-86481,3,35.99,107.97,2014-02-01 17:24:32,bronze
3,412290,Jerde-Hilpert,B1-20000,23,78.90,1814.70,2014-02-01 19:56:48,bronze
4,672390,Kuhn-Gusikowski,S1-06532,48,55.82,2679.36,2014-02-02 03:45:20,silver



Using Categories

One of the relatively new functions in pandas is support for categorical data. From the pandas, documentation -

"Categoricals are a pandas data type, which correspond to categorical variables in statistics: a variable, which can take on only a limited, and usually fixed, number of possible values (categories; levels in R). Examples are gender, social class, blood types, country affiliations, observation time or ratings via Likert scales."

For our purposes, the status field is a good candidate for a category type.

In [122]:
all_data_st["status"] = all_data_st["status"].astype("category")
all_data_st.dtypes


account number             int64
name                      object
sku                       object
quantity                   int64
unit price               float64
ext price                float64
date              datetime64[ns]
status                  category
dtype: object

Categories get more interesting when you assign order to the categories. Right now, if we call sort on the column, it will sort alphabetically. We use set_categories to tell it the order we want to use for this category object. In this case, we use the Olympic medal ordering.

In [125]:
all_data_st["status"].cat.set_categories([ "gold","silver","bronze"],inplace=True)
all_data_st.sort_values(["status"]).head()

,account number,name,sku,quantity,unit price,ext price,date,status
68,740150,Barton LLC,B1-38851,17,81.22,1380.74,2014-02-17 17:12:16,gold
63,257198,"Cronin, Oberbrunner and Spencer",S1-27722,28,10.21,285.88,2014-02-15 17:27:44,gold
207,740150,Barton LLC,B1-86481,20,30.41,608.20,2014-01-22 16:33:51,gold
61,740150,Barton LLC,B1-20000,28,81.39,2278.92,2014-02-15 07:45:16,gold
60,239344,Stokes LLC,S2-83881,30,43.00,1290.00,2014-02-15 02:13:23,gold


In [126]:
# we can look at that field in summary
all_data_st["status"].describe()

count        384
unique         3
top       bronze
freq         172
Name: status, dtype: object

For instance, if you want to take a quick look at how your top tier customers are performaing compared to the bottom. Use groupby to give us the average of the values.

In [127]:
all_data_st.groupby(["status"])["quantity","unit price","ext price"].mean()

,quantity,unit price,ext price
status,,,
gold,24.375000,53.723889,1351.944583
silver,22.842857,57.272714,1320.032214
bronze,25.616279,57.371163,1472.965930


In [128]:
# or you can run multiple aggregations

all_data_st.groupby(["status"])["quantity","unit price","ext price"].agg([np.sum,np.mean, np.std])

quantity                       unit price                        \
            sum       mean        std        sum       mean        std   
status                                                                   
gold       1755  24.375000  14.575145    3868.12  53.723889  28.740080   
silver     3198  22.842857  14.512843    8018.18  57.272714  26.556242   
bronze     4406  25.616279  14.136071    9867.84  57.371163  26.857370   

        ext price                            
              sum         mean          std  
status                                       
gold     97340.01  1351.944583  1182.657312  
silver  184804.51  1320.032214  1086.384051  
bronze  253350.14  1472.965930  1116.683843

So, what does this tell you? Well, the data is completely random but my first observation is that we sell more units to our bronze customers than gold. Even when you look at the total dollar value associated with bronze vs. gold, it looks backwards.

Maybe we should look at how many bronze customers we have and see what is going on.

What I plan to do is filter out the unique accounts and see how many gold, silver and bronze customers there are.

In [130]:
all_data_st.drop_duplicates(subset=["account number","name"]).iloc[:,[0,1,7]].groupby(["status"])["name"].count()

status
gold      4
silver    7
bronze    9
Name: name, dtype: int64

Ok. This makes a little more sense. We see that we have 9 bronze customers and only 4 customers. That is probably why the volumes are so skewed towards our bronze customers.



# Pivot tables


In [1]:
%autosave 30
import pandas as pd
import numpy as np

Autosaving every 30 seconds


Read in our sales data

In [2]:
df = pd.read_excel("sales-funnel.xlsx")
df.head()

,Account,Name,Rep,Manager,Product,Quantity,Price,Status
0,714466,Trantow-Barrows,Craig Booker,Debra Henley,CPU,1,30000,presented
1,714466,Trantow-Barrows,Craig Booker,Debra Henley,Software,1,10000,presented
2,714466,Trantow-Barrows,Craig Booker,Debra Henley,Maintenance,2,5000,pending
3,737550,"Fritsch, Russel and Anderson",Craig Booker,Debra Henley,CPU,1,35000,declined
4,146832,Kiehn-Spinka,Daniel Hilton,Debra Henley,CPU,2,65000,won


For convenience sake, let’s define the status column as a category and set the order we want to view.

This isn’t strictly required but helps us keep the order we want as we work through analyzing the data. Remember that inplace makes changes in the dataframe, not a new one. Kind of like SAS where you could sort existing dataset or create a sorted copy

In [3]:
df["Status"] = df["Status"].astype("category")
df["Status"].cat.set_categories(["won","pending","presented","declined"],inplace=True)

# Pivot the data

As we build up the pivot table, I think it’s easiest to take it one step at a time. Add items and check each step to verify you are getting the results you expect. Don’t be afraid to play with the order and the variables to see what presentation makes the most sense for your needs.

The simplest pivot table must have a dataframe and an index . In this case, let’s use the Name as our index.

In [4]:
pd.pivot_table(df,index=["Name"])

,Account,Price,Quantity
Name,,,
Barton LLC,740150.0,35000.0,1.000000
"Fritsch, Russel and Anderson",737550.0,35000.0,1.000000
Herman LLC,141962.0,65000.0,2.000000
Jerde-Hilpert,412290.0,5000.0,2.000000
"Kassulke, Ondricka and Metz",307599.0,7000.0,3.000000
Keeling LLC,688981.0,100000.0,5.000000
Kiehn-Spinka,146832.0,65000.0,2.000000
Koepp Ltd,729833.0,35000.0,2.000000
Kulas Inc,218895.0,25000.0,1.500000


You can have multiple indexes as well. In fact, most of the pivot_table args can take multiple values via a list.

In [5]:
pd.pivot_table(df,index=["Name","Rep","Manager"])

,,,Account,Price,Quantity
Name,Rep,Manager,,,
Barton LLC,John Smith,Debra Henley,740150.0,35000.0,1.000000
"Fritsch, Russel and Anderson",Craig Booker,Debra Henley,737550.0,35000.0,1.000000
Herman LLC,Cedric Moss,Fred Anderson,141962.0,65000.0,2.000000
Jerde-Hilpert,John Smith,Debra Henley,412290.0,5000.0,2.000000
"Kassulke, Ondricka and Metz",Wendy Yule,Fred Anderson,307599.0,7000.0,3.000000
Keeling LLC,Wendy Yule,Fred Anderson,688981.0,100000.0,5.000000
Kiehn-Spinka,Daniel Hilton,Debra Henley,146832.0,65000.0,2.000000
Koepp Ltd,Wendy Yule,Fred Anderson,729833.0,35000.0,2.000000
Kulas Inc,Daniel Hilton,Debra Henley,218895.0,25000.0,1.500000


This is interesting but not particularly useful. What we probably want to do is look at this by Manager and Rep. It’s easy enough to do by changing the index .

In [6]:
pd.pivot_table(df,index=["Manager","Rep"])

Account         Price  Quantity
Manager       Rep                                            
Debra Henley  Craig Booker   720237.0  20000.000000  1.250000
              Daniel Hilton  194874.0  38333.333333  1.666667
              John Smith     576220.0  20000.000000  1.500000
Fred Anderson Cedric Moss    196016.5  27500.000000  1.250000
              Wendy Yule     614061.5  44250.000000  3.000000

You can see that the pivot table is smart enough to start aggregating the data and summarizing it by grouping the reps with their managers. Now we start to get a glimpse of what a pivot table can do for us.

For this purpose, the Account and Quantity columns aren’t really useful. Let’s remove it by explicitly defining the columns we care about using the values field.

In [7]:
pd.pivot_table(df,index=["Manager","Rep"],values=["Price"])

Price
Manager       Rep                        
Debra Henley  Craig Booker   20000.000000
              Daniel Hilton  38333.333333
              John Smith     20000.000000
Fred Anderson Cedric Moss    27500.000000
              Wendy Yule     44250.000000

The price column automatically averages the data but we can do a count or a sum. Adding them is simple using aggfunc and np.sum .

In [8]:
pd.pivot_table(df,index=["Manager","Rep"],values=["Price"],aggfunc=np.sum)

Price
Manager       Rep                  
Debra Henley  Craig Booker    80000
              Daniel Hilton  115000
              John Smith      40000
Fred Anderson Cedric Moss    110000
              Wendy Yule     177000

aggfunc can take a list of functions. Let’s try a mean using the numpy mean function and len to get a count.

In [9]:
pd.pivot_table(df,index=["Manager","Rep"],values=["Price"],aggfunc=[np.mean,len])

mean   len
                                    Price Price
Manager       Rep                              
Debra Henley  Craig Booker   20000.000000     4
              Daniel Hilton  38333.333333     3
              John Smith     20000.000000     2
Fred Anderson Cedric Moss    27500.000000     4
              Wendy Yule     44250.000000     4

If we want to see sales broken down by the products, the columns variable allows us to define one or more columns.

I think one of the confusing points with the pivot_table is the use of columns and values . Remember, columns are optional - they provide an additional way to segment the actual values you care about. The aggregation functions are applied to the values you list.

In [10]:
pd.pivot_table(df,index=["Manager","Rep"],values=["Price"],
               columns=["Product"],aggfunc=[np.sum])

sum                             
                                Price                             
Product                           CPU Maintenance Monitor Software
Manager       Rep                                                 
Debra Henley  Craig Booker    65000.0      5000.0     NaN  10000.0
              Daniel Hilton  105000.0         NaN     NaN  10000.0
              John Smith      35000.0      5000.0     NaN      NaN
Fred Anderson Cedric Moss     95000.0      5000.0     NaN  10000.0
              Wendy Yule     165000.0      7000.0  5000.0      NaN

The NaN’s are a bit distracting. If we want to remove them, we could use fill_value to set them to 0.

In [11]:
pd.pivot_table(df,index=["Manager","Rep"],values=["Price"],
               columns=["Product"],aggfunc=[np.sum],fill_value=0)

sum                             
                              Price                             
Product                         CPU Maintenance Monitor Software
Manager       Rep                                               
Debra Henley  Craig Booker    65000        5000       0    10000
              Daniel Hilton  105000           0       0    10000
              John Smith      35000        5000       0        0
Fred Anderson Cedric Moss     95000        5000       0    10000
              Wendy Yule     165000        7000    5000        0

I think it would be useful to add the quantity as well. Add Quantity to the values list.

In [12]:
pd.pivot_table(df,index=["Manager","Rep"],values=["Price","Quantity"],
               columns=["Product"],aggfunc=[np.sum],fill_value=0)

sum                                        \
                              Price                              Quantity   
Product                         CPU Maintenance Monitor Software      CPU   
Manager       Rep                                                           
Debra Henley  Craig Booker    65000        5000       0    10000        2   
              Daniel Hilton  105000           0       0    10000        4   
              John Smith      35000        5000       0        0        1   
Fred Anderson Cedric Moss     95000        5000       0    10000        3   
              Wendy Yule     165000        7000    5000        0        7   

                                                          
                                                          
Product                     Maintenance Monitor Software  
Manager       Rep                                         
Debra Henley  Craig Booker            2       0        1  
              Daniel Hilton           0       0        1  
              John Smith              2       0        0  
Fred Anderson Cedric Moss             1       0        1  
              Wendy Yule              3       2        0

What’s interesting is that you can move items to the index to get a different visual representation. Remove Product from the columns and add to the index .

In [13]:
pd.pivot_table(df,index=["Manager","Rep","Product"],
               values=["Price","Quantity"],aggfunc=[np.sum],fill_value=0)

sum         
                                          Price Quantity
Manager       Rep           Product                     
Debra Henley  Craig Booker  CPU           65000        2
                            Maintenance    5000        2
                            Software      10000        1
              Daniel Hilton CPU          105000        4
                            Software      10000        1
              John Smith    CPU           35000        1
                            Maintenance    5000        2
Fred Anderson Cedric Moss   CPU           95000        3
                            Maintenance    5000        1
                            Software      10000        1
              Wendy Yule    CPU          165000        7
                            Maintenance    7000        3
                            Monitor        5000        2

For this data set, this representation makes more sense. Now, what if I want to see some totals? margins=True does that for us.

In [14]:
pd.pivot_table(df,index=["Manager","Rep","Product"],
               values=["Price","Quantity"],
               aggfunc=[np.sum,np.mean],fill_value=0,margins=True)

sum                   mean  \
                                            Price Quantity         Price   
Manager       Rep           Product                                        
Debra Henley  Craig Booker  CPU           65000.0      2.0  32500.000000   
                            Maintenance    5000.0      2.0   5000.000000   
                            Software      10000.0      1.0  10000.000000   
              Daniel Hilton CPU          105000.0      4.0  52500.000000   
                            Software      10000.0      1.0  10000.000000   
              John Smith    CPU           35000.0      1.0  35000.000000   
                            Maintenance    5000.0      2.0   5000.000000   
Fred Anderson Cedric Moss   CPU           95000.0      3.0  47500.000000   
                            Maintenance    5000.0      1.0   5000.000000   
                            Software      10000.0      1.0  10000.000000   
              Wendy Yule    CPU          165000.0      7.0  82500.000000   
                            Maintenance    7000.0      3.0   7000.000000   
                            Monitor        5000.0      2.0   5000.000000   
All                                      522000.0     30.0  30705.882353   

                                                   
                                         Quantity  
Manager       Rep           Product                
Debra Henley  Craig Booker  CPU          1.000000  
                            Maintenance  2.000000  
                            Software     1.000000  
              Daniel Hilton CPU          2.000000  
                            Software     1.000000  
              John Smith    CPU          1.000000  
                            Maintenance  2.000000  
Fred Anderson Cedric Moss   CPU          1.500000  
                            Maintenance  1.000000  
                            Software     1.000000  
              Wendy Yule    CPU          3.500000  
                            Maintenance  3.000000  
                            Monitor      2.000000  
All                                      1.764706

Let’s move the analysis up a level and look at our pipeline at the manager level. Notice how the status is ordered based on our earlier category definition.

In [15]:
pd.pivot_table(df,index=["Manager","Status"],values=["Price"],
               aggfunc=[np.sum],fill_value=0,margins=True)

sum
                            Price
Manager       Status             
Debra Henley  won         65000.0
              pending     50000.0
              presented   50000.0
              declined    70000.0
Fred Anderson won        172000.0
              pending      5000.0
              presented   45000.0
              declined    65000.0
All                      522000.0

A really handy feature is the ability to pass a dictionary to the aggfunc so you can perform different functions on each of the values you select. This has a side-effect of making the labels a little cleaner.

In [16]:
pd.pivot_table(df,index=["Manager","Status"],columns=["Product"],values=["Quantity","Price"],
               aggfunc={"Quantity":len,"Price":np.sum},fill_value=0)

Price                              Quantity  \
Product                     CPU Maintenance Monitor Software      CPU   
Manager       Status                                                    
Debra Henley  won         65000           0       0        0        1   
              pending     40000       10000       0        0        1   
              presented   30000           0       0    20000        1   
              declined    70000           0       0        0        2   
Fred Anderson won        165000        7000       0        0        2   
              pending         0        5000       0        0        0   
              presented   30000           0    5000    10000        1   
              declined    65000           0       0        0        1   

                                                      
Product                 Maintenance Monitor Software  
Manager       Status                                  
Debra Henley  won                 0       0        0  
              pending             2       0        0  
              presented           0       0        2  
              declined            0       0        0  
Fred Anderson won                 1       0        0  
              pending             1       0        0  
              presented           0       1        1  
              declined            0       0        0

You can provide a list of aggfunctions to apply to each value too:

In [17]:
table = pd.pivot_table(df,index=["Manager","Status"],columns=["Product"],values=["Quantity","Price"],
               aggfunc={"Quantity":len,"Price":[np.sum,np.mean]},fill_value=0)
table

Price                                       \
                          mean                                  sum   
Product                    CPU Maintenance Monitor Software     CPU   
Manager       Status                                                  
Debra Henley  won        65000           0       0        0   65000   
              pending    40000        5000       0        0   40000   
              presented  30000           0       0    10000   30000   
              declined   35000           0       0        0   70000   
Fred Anderson won        82500        7000       0        0  165000   
              pending        0        5000       0        0       0   
              presented  30000           0    5000    10000   30000   
              declined   65000           0       0        0   65000   

                                                     Quantity              \
                                                          len               
Product                 Maintenance Monitor Software      CPU Maintenance   
Manager       Status                                                        
Debra Henley  won                 0       0        0        1           0   
              pending         10000       0        0        1           2   
              presented           0       0    20000        1           0   
              declined            0       0        0        2           0   
Fred Anderson won              7000       0        0        2           1   
              pending          5000       0        0        0           1   
              presented           0    5000    10000        1           0   
              declined            0       0        0        1           0   

                                          
                                          
Product                 Monitor Software  
Manager       Status                      
Debra Henley  won             0        0  
              pending         0        0  
              presented       0        2  
              declined        0        0  
Fred Anderson won             0        0  
              pending         0        0  
              presented       1        1  
              declined        0        0

# Advanced Pivot Table Filtering

Once you have generated your data, it is in a DataFrame so you can filter on it using your standard DataFrame functions.

If you want to look at just one manager:

In [18]:
table.query('Manager == ["Debra Henley"]')

Price                                                  \
                         mean                                 sum               
Product                   CPU Maintenance Monitor Software    CPU Maintenance   
Manager      Status                                                             
Debra Henley won        65000           0       0        0  65000           0   
             pending    40000        5000       0        0  40000       10000   
             presented  30000           0       0    10000  30000           0   
             declined   35000           0       0        0  70000           0   

                                        Quantity                               
                                             len                               
Product                Monitor Software      CPU Maintenance Monitor Software  
Manager      Status                                                            
Debra Henley won             0        0        1           0       0        0  
             pending         0        0        1           2       0        0  
             presented       0    20000        1           0       0        2  
             declined        0        0        2           0       0        0

We can look at all of our pending and won deals.

In [19]:
table.query('Status == ["pending","won"]')

Price                                                   \
                        mean                                  sum               
Product                  CPU Maintenance Monitor Software     CPU Maintenance   
Manager       Status                                                            
Debra Henley  won      65000           0       0        0   65000           0   
              pending  40000        5000       0        0   40000       10000   
Fred Anderson won      82500        7000       0        0  165000        7000   
              pending      0        5000       0        0       0        5000   

                                       Quantity                               
                                            len                               
Product               Monitor Software      CPU Maintenance Monitor Software  
Manager       Status                                                          
Debra Henley  won           0        0        1           0       0        0  
              pending       0        0        1           2       0        0  
Fred Anderson won           0        0        2           1       0        0  
              pending       0        0        0           1       0        0